[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FeetCodingHommy/kakao-arena-melon-playlist-dataset-and-more/blob/master/colab_notebooks/crawl_test.ipynb)

# 세팅

In [ ]:
!git clone https://github.com/FeetCodingHommy/kakao-arena-melon-playlist-dataset-and-more.git

In [ ]:
!pip install -qq scrapy
!pip install -qq selenium

In [ ]:
import sys
sys.path.append("/content/kakao-arena-melon-playlist-dataset-and-more/melon_scrapy") # custom_settings에 필요
import melon_scrapy

In [ ]:
# 코랩 세션 끊김 방지
#   아래 코드를 크롬 F12 콘솔에 실행

%%javascript
// function ClickConnect(){
//     console.log("9분!"); 
//     document.querySelector("colab-toolbar-button").click() 
// }
// setInterval(ClickConnect, 9 * 60 * 1000)
console.log("Melon Crawler Test")

  * 서버에 바로 다운 받을 수 있는 API는 제공하지 않음 ([관련](https://arena.kakao.com/forum/topics/193))
    * 직접 다운해서 구글 드라이브에 올리고 마운트 시키는 방법
    * 임시로 구글 드라이브에 있는 파일을 공유해서 다운받는 방법

In [ ]:
# 임시로 구글 드라이브에 있는 파일을 공유해서 다운받는 방법 - 나중에 링크 비활성화 필수

!gdown --id 1JKxtYManLah1kZ6C0KqlTeqn0uKUQAc7
!mv /content/song_meta.json /content/kakao-arena-melon-playlist-dataset-and-more/_scrap_input/

# 멜론 곡 별 앨범 ID 정보

In [ ]:
from melon_scrapy.my_config import COLAB_SCRAPY_INPUT_PATH
import json
import os

with open(os.path.join(COLAB_SCRAPY_INPUT_PATH, "song_meta.json"), mode='r') as json_file:
    all_songs = json.load(json_file)

all_songs[0]

In [ ]:
%%time

list_of_dicts = list()
for song in all_songs:
    list_of_dicts.append({
        "album_id": song["album_id"],
        "id": song["id"],
        "song_name": song["song_name"]
    })

print(len(list_of_dicts))

In [ ]:
import pandas as pd

df = pd.DataFrame(list_of_dicts)
del list_of_dicts
df.to_csv("song_meta_all_ids.csv", encoding="utf-8", index=None)

In [ ]:
df.head()

# 크롤링 연습
  * ※ robots.txt(로봇 배제 표준)을 준수하는 것이 웬만해선 좋음
    * 관련 글 1: [개인 블로그 글](https://toyrit.com/robots-txt-set/)
    * 관련 글 2: [티스토리 글](https://heodolf.tistory.com/9)(아래에 또 등장)
      * ```ROBOTSTXT_OBEY = True```하면 준수하는 것
    * 관련 글 3: [개인 블로그 글](https://technote.luminance.kr/entry/UserAgents)
      * 유명 UserAgent 정리
    * 관련 글 4: [해외 기사](https://www.zyte.com/blog/how-to-crawl-the-web-politely-with-scrapy/) - 아 님들 scrapy로 무단으로 긁어갈꺼면 최소 매너 좀;
  * 멜론의 [robots.txt](https://www.melon.com/robots.txt)
    * User-agent: Googlebot = 구글
    * User-agent: Twitterbot
    * User-agent: Applebot
    * User-agent: Yeti = 네이버
    * User-agent: daumoa / daumoa-image = 다음
      * 다음카카오 쪽에 권한이 제일 많음
    * 기타 - ```Disallow: /``` (= 다 긁어가지마)
      * ㄷㄷ...

## 1. 곡 하나 긁어보기
  * heoDolf님 블로그 글: ([링크](https://heodolf.tistory.com/9))

In [ ]:
!scrapy runspider /content/kakao-arena-melon-playlist-dataset-and-more/melon_scrapy/melon_scrapy/spiders/test_spider_1_crawl_one_song.py

## 2. 앨범 링크로부터 맞는 곡 찾아와서 긁어보기
※ 조진 점) 3일간의 삽질과 창희에게 SOS를 친 결과, scrapy가 멜론에서 곡들이 다 뜨기 전에 긁어와버린다는 걸 알게 되었다.
  * ```response.css("a").getall()```를 해보면 딱 앨범 정보 전까지의 모든 ```<a>``` 태그들만 다 긁어온다는 것을 알 수 있다.
    * 창희 추천 글: 동기 vs 비동기 ([링크](https://webclub.tistory.com/605))
  * \[StackOverflow\] scrapy가 페이지 다 로딩 될때까지 기다릴 수 있나염? - ㄴㄴ 안됨요 ㅅㄱ ([링크](https://stackoverflow.com/questions/31330890/waiting-for-the-loading-page-with-scrapy))
  * \[블로그 글\] scrapy와 selenium을 같이 쓰시면 됩니다 ㅎㅎ ([링크](https://www.hanumoka.net/2020/07/11/python-20200711-python-scrapy-selenium/))
    * 아 BeautifulSoup랑 Selenium 환경설정도 귀찮고 그냥 안써볼라고 scrapy 쓴건데

In [ ]:
!scrapy runspider /content/kakao-arena-melon-playlist-dataset-and-more/melon_scrapy/melon_scrapy/spiders/test_spider_2_crawl_song_from_album.py